<a href="https://colab.research.google.com/github/AzazelSarie/Kaggle/blob/main/Autogluon_for_Car_Price_Regression_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install autogluon
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.9 MB/s eta 0:00:00


In [30]:
from autogluon.tabular import TabularPredictor
import pandas as pd
import optuna

In [14]:
data = pd.read_csv('/content/train.csv')
data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [38]:
# Define multiple configurations for each model under the correct keys
#[I 2024-09-23 08:36:39,443] Trial 5 finished with value: -67565.84834616985 and parameters: {'gbm1_num_boost_round': 221, 'gbm1_learning_rate': 0.16382878552862984, 'gbm1_max_depth': 8, 'gbm2_num_boost_round': 647, 'gbm2_learning_rate': 0.02396945525183155, 'gbm2_max_depth': 9, 'xgb1_n_estimators': 251, 'xgb1_learning_rate': 0.010758581805030776, 'xgb1_max_depth': 5, 'xgb2_n_estimators': 783, 'xgb2_learning_rate': 0.1821589653782948, 'xgb2_max_depth': 9, 'cat1_border_count': 10, 'cat1_learning_rate': 0.024476464940931884, 'cat1_depth': 20, 'cat2_learning_rate': 0.09797563785687428, 'cat2_depth': 9, 'rf1_n_estimators': 101, 'rf1_max_depth': 15, 'rf2_n_estimators': 733, 'rf2_max_depth': 28, 'nn_epochs': 1283, 'nn_learning_rate': 0.007480576722905295, 'nn_batch_size': 31, 'nn_layers': [300, 200, 100], 'nn_dropout': 0.43502503008304694}. Best is trial 5 with value: -67565.84834616985.

hyperparameters = {
    'GBM': [
        {'gbm1_num_boost_round': 221, 'gbm1_learning_rate': 0.16382878552862984, 'gbm1_max_depth': 8},
        {'gbm2_num_boost_round': 647, 'gbm2_learning_rate': 0.02396945525183155, 'gbm2_max_depth': 9},
    ],
    'XGB': [
        {'xgb1_n_estimators': 251, 'xgb1_learning_rate': 0.010758581805030776, 'xgb1_max_depth': 5},
        {'xgb2_n_estimators': 783, 'xgb2_learning_rate': 0.1821589653782948, 'xgb2_max_depth': 9},
    ],
    'CAT': [
        {'cat1_border_count': 10, 'cat1_learning_rate': 0.024476464940931884, 'cat1_depth': 20},
        {'cat2_learning_rate': 0.09797563785687428, 'cat2_depth': 9},
    ],
    'RF': [
        {'rf1_n_estimators': 101, 'rf1_max_depth': 15},
        {'rf2_n_estimators': 733, 'rf2_max_depth': 28},
    ],
    'NN_TORCH': {'nn_epochs': 10000, 'nn_learning_rate': 0.007480576722905295, 'nn_batch_size': 31, 'nn_layers': [1000, 800, 600, 400, 200, 100, 50, 25, 5, 1], 'nn_dropout': 0.43502503008304694}
}

# Train the models with stacking
predictor = TabularPredictor(label='price').fit(
    data,
    presets='best_quality',  # Preset to maximize performance
    hyperparameters=hyperparameters,
    num_bag_folds=10,        # Number of bagging folds
    num_stack_levels=5       # Increased stack levels
)

# Evaluate performance
performance = predictor.evaluate(data)
print(f"Model performance: {performance}")


No path specified. Models will be saved in: "AutogluonModels/ag-20240923_151527"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       9.97 GB / 12.67 GB (78.7%)
Disk Space Avail:   181.01 GB / 225.83 GB (80.2%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=5, num_bag_folds=10, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will b

Model performance: {'root_mean_squared_error': -70145.76865650505, 'mean_squared_error': -4920428860.411926, 'mean_absolute_error': -19396.80633907799, 'r2': 0.20797723547155111, 'pearsonr': 0.4730434805503562, 'median_absolute_error': -9654.927734375}


In [ ]:
# Define the Optuna objective function
def objective(trial):
    # Define different hyperparameters for each GBM model configuration
    gbm_params1 = {
        'num_boost_round': trial.suggest_int('gbm1_num_boost_round', 100, 500),  # Higher learning rate, fewer rounds
        'learning_rate': trial.suggest_float('gbm1_learning_rate', 0.1, 0.2),
        'max_depth': trial.suggest_int('gbm1_max_depth', 3, 10)  # Shallower trees for high LR
    }
    gbm_params2 = {
        'num_boost_round': trial.suggest_int('gbm2_num_boost_round', 500, 1000),  # Lower learning rate, more rounds
        'learning_rate': trial.suggest_float('gbm2_learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('gbm2_max_depth', 5, 15)  # Deeper trees for low LR
    }

    # Define different hyperparameters for each XGB model configuration
    xgb_params1 = {
        'n_estimators': trial.suggest_int('xgb1_n_estimators', 100, 500),  # Lower LR, fewer estimators
        'learning_rate': trial.suggest_float('xgb1_learning_rate', 0.01, 0.15),
        'max_depth': trial.suggest_int('xgb1_max_depth', 3, 10)  # Shallower trees for lower LR
    }
    xgb_params2 = {
        'n_estimators': trial.suggest_int('xgb2_n_estimators', 500, 1000),  # Higher LR, more estimators
        'learning_rate': trial.suggest_float('xgb2_learning_rate', 0.15, 0.3),
        'max_depth': trial.suggest_int('xgb2_max_depth', 5, 15)  # Deeper trees for higher LR
    }

    # Define different hyperparameters for each CatBoost model configuration
    cat_params1 = {
        'border_count': trial.suggest_int('cat1_border_count', 5, 20),
        'learning_rate': trial.suggest_float('cat1_learning_rate', 0.01, 0.05),  # Lower LR for deeper trees
        'depth': trial.suggest_int('cat1_depth', 10, 25)
    }
    cat_params2 = {
        'border_count': trial.suggest_int('cat1_border_count', 5, 20),
        'learning_rate': trial.suggest_float('cat2_learning_rate', 0.05, 0.1),  # Higher LR for shallower trees
        'depth': trial.suggest_int('cat2_depth', 7, 20)
    }

    # Define hyperparameters for Random Forest (you can repeat this for multiple configurations if needed)
    rf_params1 = {
        'n_estimators': trial.suggest_int('rf1_n_estimators', 100, 500),  # Fewer trees for smaller depth
        'max_depth': trial.suggest_int('rf1_max_depth', 3, 20)
    }
    rf_params2 = {
        'n_estimators': trial.suggest_int('rf2_n_estimators', 500, 1000),  # More trees for larger depth
        'max_depth': trial.suggest_int('rf2_max_depth', 10, 30)
    }

    # Combine into hyperparameter dictionary for AutoGluon
    hyperparameters = {
        'GBM': [gbm_params1, gbm_params2],
        'XGB': [xgb_params1, xgb_params2],
        'CAT': [cat_params1, cat_params2],
        'RF': [rf_params1, rf_params2],
        'NN_TORCH': {
        'num_epochs': trial.suggest_int('nn_epochs', 1000, 2000),
        'learning_rate': trial.suggest_float('nn_learning_rate', 0.0001, 0.01),
        'batch_size': trial.suggest_int('nn_batch_size', 16, 64),
        'layers': trial.suggest_categorical('nn_layers', [[100, 50], [200, 100, 50], [300, 200, 100]]),
        'dropout': trial.suggest_float('nn_dropout', 0.1, 0.5)
        }  # PyTorch-based neural network
    }

    # Train the models with AutoGluon
    predictor = TabularPredictor(label='price').fit(
        train_data=data,
        presets='best_quality',  # Preset to maximize performance
        hyperparameters=hyperparameters,
        num_bag_folds=10,        # Number of bagging folds
        num_stack_levels=5,      # Increased stack levels
        verbosity=0              # Reduce output verbosity
    )

    # Evaluate performance (e.g., root mean squared error)
    performance = predictor.evaluate(data)['root_mean_squared_error']

    # Return the evaluation metric (Optuna tries to minimize this value)
    return performance

# Run Optuna study to optimize the hyperparameters
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # Run 50 trials to optimize

# Print the best hyperparameters
print(f"Best hyperparameters: {study.best_params}")

# Access the best hyperparameters after tuning
best_params = study.best_params


In [39]:
import pandas as pd

# Load the test data (make sure it contains 'id' and the relevant feature columns)
test_data = pd.read_csv('test.csv')

# Generate predictions for the 'price' column
predictions = predictor.predict(test_data)

# Create a submission DataFrame with 'id' and predicted 'price'
submission = pd.DataFrame({
    'id': test_data['id'],  # Assuming 'id' is a column in the test data
    'price': predictions     # Predicted price from AutoGluon
})




In [41]:
# Save the submission file as a CSV (without index)
submission.to_csv('submission v4.csv', index=False)